# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [56]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

In [57]:
#Your code here
df = pd.read_csv('homepage_actions.csv')

In [11]:
df.head(3)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view


In [28]:
df.groupby('action').size()  # 1860 click and 6328 view 
6328 - 1860 # didn't click 

4468

In [12]:
exp = df[df.group == 'experiment']
control = df[df.group=='control']

In [15]:
exp.groupby('action').size()

action
click     928
view     2996
dtype: int64

In [16]:
control.groupby('action').size()

action
click     932
view     3332
dtype: int64

In [31]:
# check for common users for either experiemnt and control 
eids = set(df[df.group=='experiment']['id'].unique())
cids = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(eids.intersection(cids))))

Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [62]:
df['count'] = 1
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)
control.head(), experiment.head()

(action  click  view
 id                 
 182994    1.0   1.0
 183089    0.0   1.0
 183248    1.0   1.0
 183515    0.0   1.0
 183524    0.0   1.0, action  click  view
 id                 
 182988    0.0   1.0
 183136    0.0   1.0
 183141    1.0   1.0
 183283    0.0   1.0
 183389    0.0   1.0)

In [65]:
import statsmodels.stats.weightstats as wt

In [66]:
wt.ttest_ind(experiment.click, control.click, alternative='larger')  # appear to reject h0

(2.6195696844542193, 0.004412549457479159, 6326.0)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [37]:
import scipy.stats as st

In [74]:
#Your code here
p = control.click.mean()
expected_exp = p*len(experiment)
std = np.sqrt(len(experiment)*p*(1-p))
std, p, expected_exp

(24.568547907005815, 0.2797118847539016, 838.0168067226891)

In [76]:
z_score = (experiment.click.sum()-expected_exp)/std
st.norm.sf(z_score)  # > alpha , fail to reject null hypothesis 

0.00012486528006951198

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.